* version check

In [2]:
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)

1.12.1+cu102
0.13.1+cu102


In [3]:
!which python

/opt/conda/bin/python


## import

In [4]:
from Nb_utils import *
import copy
import random
import torch.nn.functional as F

/home/superorange5/.local/lib/python3.7/site-packages/torchvision/transforms/transforms.py:892: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  "Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. "


In [5]:
    def resize(model, data):
        data = copy.deepcopy(data)
        bs = len(data)
        for i in range(bs):
            img = data[i]['image']
            h, w = img.shape[-2], img.shape[-1]
            ratio = random.uniform(0.5, 1.0)
            d_h, d_w = int(h * ratio), int(w * ratio)
            x1 = int((w - d_w) / 2)
            y1 = int((h - d_h) / 2)
            bg = torch.zeros_like(img)
            try:
                bg += model.pixel_mean.cpu().int()
            except:
                bg += model.module.pixel_mean.cpu().int()
            bg[:, y1:y1 + d_h, x1:x1 + d_w] = F.interpolate(img.unsqueeze(0).float(),
                                                            size=(d_h, d_w),
                                                            align_corners=False,
                                                            mode='bilinear').squeeze(0)
            data[i]['image'] = bg
            if data[i]['instances'].has('gt_boxes'):
                data[i]['instances'].gt_boxes.tensor *= ratio
                data[i]['instances'].gt_boxes.tensor[:, 0] += x1
                data[i]['instances'].gt_boxes.tensor[:, 2] += x1
                data[i]['instances'].gt_boxes.tensor[:, 1] += y1
                data[i]['instances'].gt_boxes.tensor[:, 3] += y1

            if data[i]['instances'].has('pseudo_boxes'):
                data[i]['instances'].pseudo_boxes.tensor *= ratio
                data[i]['instances'].pseudo_boxes.tensor[:, 0] += x1
                data[i]['instances'].pseudo_boxes.tensor[:, 2] += x1
                data[i]['instances'].pseudo_boxes.tensor[:, 1] += y1
                data[i]['instances'].pseudo_boxes.tensor[:, 3] += y1
        return data

## load model

### load local previous

In [6]:
#LOCAL_PREVIOUS:c2b_source-only 
#    "keep_experiments/c2b_source-only/model_final.pth"
local_model_previous_path = "c2b_source-only"
local_previous_model = load_sourceonlyModel(local_model_previous_path,'final')

keep_experiments/c2b_source-only/cfg.yaml
keep_experiments/c2b_source-only/model_final.pth
-------- pretrained model loaded ---------


### load global

In [7]:
global_model_path = "output/FedAvg_ck2b/FedAvg_0_20230116.pth" 
cfg = setup("keep_experiments/c2b/cfg.yaml")
Trainer =DefaultTrainer
global_model = Trainer.build_model(cfg)    
DetectionCheckpointer(global_model).resume_or_load(global_model_path, resume=False)

-------- pretrained model loaded ---------


{'matching_heuristics': True}

* check dataset

In [43]:
cfg['DATASETS']['TRAIN_LABEL']

('VOC2007_citytrain1',)

### copy global to local

In [44]:
net = copy.deepcopy(global_model)


## load training data

In [45]:
from pt.data.build import build_detection_sourceonly_loader_two_crops
from pt.data.dataset_mapper import DatasetMapperTwoCropSeparate

mapper = DatasetMapperTwoCropSeparate(cfg, True)
train_data_loader = build_detection_sourceonly_loader_two_crops(cfg, mapper)

In [52]:
data_loader_iter = iter(train_data_loader)
train_data =  next(data_loader_iter)

In [53]:
label_data_q, label_data_k = train_data

In [54]:
label_data_q[0]


{'file_name': 'data/VOC2007_citytrain/JPEGImages/hanover_000000_030781_leftImg8bit.jpg',
 'image_id': 'hanover_000000_030781_leftImg8bit',
 'height': 1024,
 'width': 2048,
 'instances': Instances(num_instances=3, image_height=600, image_width=1200, fields=[gt_boxes: Boxes(tensor([[  0.0000, 258.3984,  41.6016, 308.7891],
         [544.9219, 228.5156, 646.2891, 308.2031],
         [286.5234, 242.5781, 447.0703, 298.2422]])), gt_classes: tensor([0, 0, 0])]),
 'image': tensor([[[  2,  10,  34,  ..., 106,  86,  74],
          [  2,  10,  33,  ..., 123, 104,  94],
          [  2,  10,  32,  ..., 135, 118, 108],
          ...,
          [ 41,  41,  40,  ...,  42,  42,  42],
          [ 39,  38,  36,  ...,  42,  42,  42],
          [ 36,  35,  33,  ...,  42,  42,  42]],
 
         [[  5,  17,  47,  ..., 177, 171, 166],
          [  5,  17,  46,  ..., 178, 172, 168],
          [  5,  17,  45,  ..., 178, 170, 166],
          ...,
          [ 37,  36,  35,  ...,  37,  37,  37],
          [ 35,  

In [49]:
label_data_k[0]

{'file_name': 'data/VOC2007_citytrain/JPEGImages/hamburg_000000_014940_leftImg8bit.jpg',
 'image_id': 'hamburg_000000_014940_leftImg8bit',
 'height': 1024,
 'width': 2048,
 'instances': Instances(num_instances=9, image_height=600, image_width=1200, fields=[gt_boxes: Boxes(tensor([[  85.5469,  275.3906,  155.2734,  295.8984],
         [ 159.3750,  274.2188,  222.0703,  294.1406],
         [ 219.7266,  277.1484,  265.4297,  291.7969],
         [   0.0000,  246.0938,   12.3047,  294.1406],
         [ 902.3438,  254.8828,  924.0234,  267.7734],
         [ 534.9609,  264.2578,  650.9766,  301.7578],
         [ 673.8281,  258.9844,  804.4922,  298.8281],
         [1071.6797,  242.5781, 1200.0000,  291.7969],
         [ 894.7266,  251.9531, 1027.1484,  294.1406]])), gt_classes: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0])]),
 'image': tensor([[[ 18,  18,  19,  ...,   4,   2,   2],
          [ 15,  16,  16,  ...,   3,   1,   6],
          [  5,   5,   6,  ...,   2,   0,  12],
          ...,
          [

In [55]:
len(label_data_q)

16

In [56]:
label_data_q.extend(label_data_k)


In [57]:
import copy
import random
import torch.nn.functional as F

label_data_q = resize(global_model,label_data_q)

## set model mode

In [58]:
global_model.eval()
local_previous_model.eval()

GuassianGeneralizedRCNN(
  (backbone): VGG(
    (vgg_block1): Sequential(
      (0): VGGBlock(
        (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (vgg_block2): Sequential(
      (0): VGGBlock(
        (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      )
    )
    (vgg_block3): Sequential(
      (0): VGGBlock(
        (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)

In [59]:
#global_model.roi_heads.box_predictor.cls_score.weight

* check model mode

In [60]:
global_model.training


False

* make model req_grad=false

In [61]:
for param in global_model.parameters():
    param.requires_grad = False
#global_w = global_model.state_dict()

In [62]:
for param in local_previous_model.parameters():
    param.requires_grad = False



## call model and get loss

In [21]:
from detectron2.utils.events import EventStorage
with EventStorage() as storage:
    #record_dict_global, _, _, _ = global_model(label_data_q, branch="supervised")
    #record_dict_local_prev, _, _, _ = local_previous_model(label_data_q, branch="supervised")
    record_dict_local_curr, _, _, _ = net(label_data_q, branch="supervised")

/home/superorange5/.local/lib/python3.7/site-packages/torch/functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [22]:

A = global_model(label_data_q, branch="unsup_data_weak_test")

In [23]:
features,proposals_rpn, proposals_roih, ROI_predictions = net(label_data_q, branch="unsup_data_weak_test")

In [29]:
## 非必要，僅解決 CUDA out of memory. 問題
torch.cuda.empty_cache() 
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:2048"

In [30]:
with torch.no_grad():
    (_,  proposals_rpn_unsup_k, proposals_roih_unsup_k, _,)= net(label_data_q, branch="unsup_data_weak")

RuntimeError: CUDA out of memory. Tried to allocate 5.49 GiB (GPU 0; 31.75 GiB total capacity; 27.16 GiB already allocated; 761.75 MiB free; 29.86 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [31]:
features

{'vgg_block5': tensor([[[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           ...,
           [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00]],
 
          [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
            0.0000e+00, 0.0000e+00],
           ...,
           [0.0000e+00, 

In [32]:
A[4]

{'instances': Instances(num_instances=12, image_height=1024, image_width=2048, fields=[pred_boxes: Boxes(tensor([[1559.0526,  369.2750, 2028.0588,  741.8217],
         [1138.9521,  415.2973, 1202.7600,  464.2725],
         [ 943.8898,  409.7549, 1006.0596,  458.2784],
         [1865.1520,  489.8351, 2023.8794,  841.3592],
         [ 986.3124,  399.5432, 1025.6343,  427.7608],
         [1776.2692,  395.2498, 2014.1421,  804.3880],
         [ 969.1602,  401.7174, 1005.6224,  428.4144],
         [1032.2039,  412.1666, 1138.4435,  467.3717],
         [ 976.0252,  406.6786, 1022.8869,  445.2130],
         [1890.3911,  437.9253, 2028.6399,  604.5677],
         [1201.3131,  435.5332, 1237.6864,  461.1324],
         [1007.9521,  394.7413, 1046.1382,  424.4991]], device='cuda:0')), scores: tensor([0.9356, 0.8825, 0.8813, 0.8382, 0.2671, 0.2084, 0.1407, 0.0478, 0.0400,
         0.0238, 0.0238, 0.0230], device='cuda:0'), pred_classes: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'),

In [33]:
global_model.backbone(label_data_q)


AttributeError: 'list' object has no attribute 'numel'

* ### TODO!!! take feature to do similarity ###

In [34]:
cos=torch.nn.CosineSimilarity(dim=-1)


### loss1

In [36]:
record_dict_local_curr

{'loss_cls': tensor(0.0493, device='cuda:0', grad_fn=<MulBackward0>),
 'loss_box_reg': tensor(0.4126, device='cuda:0', grad_fn=<MulBackward0>),
 'loss_rpn_cls': tensor(0.0758, device='cuda:0', grad_fn=<MulBackward0>),
 'loss_rpn_loc': tensor(0.3837, device='cuda:0', grad_fn=<MulBackward0>)}

In [35]:
loss_dict = {}
for key in record_dict_local_curr.keys():
    if key.split('_')[-1] == "adv":
        loss_dict[key] = record_dict_local_curr[key] * 0.0
    elif key[:4] == "loss":
        loss_dict[key] = record_dict_local_curr[key] * 1.0
losses = sum(loss_dict.values())

In [37]:
losses

tensor(0.9214, device='cuda:0', grad_fn=<AddBackward0>)

In [38]:

optimizer = torch.optim.SGD(net.parameters(), lr = 0.01, momentum=0.9)
optimizer.zero_grad()  
losses.backward()
optimizer.step()


In [39]:
record_dict_local_prev

NameError: name 'record_dict_local_prev' is not defined

## appendix

### moon code

In [40]:
            posi = cos(pro1, pro2)
            logits = posi.reshape(-1,1)

            for previous_net in previous_nets:
                previous_net.cuda()
                _, pro3, _ = previous_net(x)
                nega = cos(pro1, pro3)
                logits = torch.cat((logits, nega.reshape(-1,1)), dim=1)

                previous_net.to('cpu')

            logits /= temperature
            labels = torch.zeros(x.size(0)).cuda().long()

            loss2 = mu * criterion(logits, labels)


            loss1 = criterion(out, target)
            loss = loss1 + loss2

NameError: name 'pro1' is not defined

### iteration

In [41]:

dataset_name = 'VOC2007_citytrain'
train_data_loader = build_train_loader(cfg, dataset_name)
for idx, inputs in enumerate(train_data_loader):    

    record_dict, _, _, _ = global_model(inputs, branch="supervised")

NameError: name 'build_train_loader' is not defined

###  call model and get loss

In [42]:
record_dict, _, _, _ = self.model(label_data_q, branch="supervised")

NameError: name 'self' is not defined